# Notebook for exploring the Chinese-English Buddhist Dictionary

In [34]:
import re

import pandas as pd
import numpy as np

import curation_util
from korean import getkoreanid
from taisho import geturl
from taisho import saveScrolls

# Load words table
colnames = ['id', 'simplified', 'traditional', 'pinyin', 
            'english', 'grammar', 'concept_cn', 'concept_en', 
            'domain_cn', 'domain_en', 'subdomain_cn', 'subdomain_en',
            'image', 'mp3', 'notes', 'headword']
types = {'id': np.uint32, 'simplified': unicode, 'traditional': unicode, 
        'pinyin': np.object, 'english': np.object, 'grammar': np.object,
        'concept_cn': np.object, 'concept_en': np.object,
        'domain_cn': np.object, 'domain_en': np.object, 'subdomain_cn': np.object,
         'subdomain_en': np.object, 'image': np.object, 'mp3': np.object,
         'notes': np.object, 'headword': np.uint32}
index_col = ['headword', 'id']
df = pd.read_table('../data/dictionary/words.txt', names=colnames, dtype=types,
                  index_col = index_col)
cols = ["simplified", "traditional", "pinyin", "english", "notes"]
df[df.simplified == '脡'][cols]

simplified traditional pinyin                       english  \
headword id                                                                  
46847    46847          脡          \N   tǐng  strips of dried meat / jerky   
         46863          脡          \N   tǐng            stiff and straight   

                       notes  
headword id                   
46847    46847  (Unihan '脡')  
         46863  (Unihan '脡')

In [35]:
# Load an entry from the Taisho canon
# Input
tid = "1735"

# Generated
entry = curation_util.GetEntry(tid)
title_cn = entry["title"]
print title_cn


Translator 澄觀 not in dictionary
GetTranslatorEn: Translator '澄觀' not found
Translator  not in dictionary
GetTranslatorEn: Translator '' not found
大方廣佛華嚴經疏


In [36]:
# Conversions for a word based on a traditional characters
title_cn = u"澄觀"
sanskrit = u""
english = u""
grammar = "proper noun"
#concept = u"\\N\t\\N"
#concept = u"书名\tBook Title"
#concept = u"论\tTreatise"
concept = u"经疏\tSūtra Commentary"
#concept = u"佛\tBuddha"
#concept = u"菩萨\tBodhisattva"
#concept = u"经\tSūtra"
#concept = u"陀罗尼\tDhāraṇī"
#subdomain = u"中国佛教\tChinese Buddhism"
subdomain = u"\\N\t\\N"
#subdomain = u"大乘佛教\tMahāyāna Buddhism"
#subdomain = u"密教\tEsoteric Buddhism"
subdomain = u"中国佛教\tChinese Buddhism"

# Generated
luid = df.count()[0] + 1 #np.amax(df.index.values)[0] + 1
print luid
tradArr = curation_util.ExtractWords(title_cn)
newWord = True
if len(tradArr) == 1:
  print "Existing word"
  newWord = False
else:
  print "New word"
domain = u"佛教\tBuddhism"
#domain = u"古文\tClassical Chinese"
traditional = ""
simplified = ""
pinyin = ""
for t in tradArr:
  sDF = df[df.traditional == t]['simplified']
  tDF = df[df.traditional == t]['traditional']
  pDF = df[df.traditional == t]['pinyin']
  if sDF.count() > 0:
    simplified += sDF.iloc[0].decode('utf-8')
    traditional +=tDF.iloc[0].decode('utf-8')
    pinyin += pDF.iloc[0].decode('utf-8') + " "
  else:
    sDF = df[df.simplified == t]['simplified']
    tDF = df[df.simplified == t]['simplified']
    pDF = df[df.simplified == t]['pinyin']
    if sDF.count() > 0:
      simplified += sDF.iloc[0].decode('utf-8')
      traditional += sDF.iloc[0].decode('utf-8')
      pinyin += pDF.iloc[0].decode('utf-8') + " "
    else:
      print "%s not found" % t
pinyin = pinyin.strip()
print simplified
trad = traditional
if simplified == traditional:
  trad = "\\N"
if grammar == "proper noun":
  pinyin = pinyin.title()

# Do some guessing on the best English name
title_en = u""
if english == "":
  english = curation_util.P2englishPN(pinyin)
if sanskrit != "":
  if english != "":
    english, title_en = u"%s / %s" % (sanskrit, english), u"%s (%s)" % (sanskrit, english)
  else:
    english = u"%s" % sanskrit
if title_en == "":
  title_en = english

print trad
print pinyin
print english

49829
New word
澄观
澄觀
Chéng Guān
Cheng Guan


In [37]:
# Template for a dictionary entry for a Buddhist term
# Input
colophon_cn = u"""第 %d 冊　No. %s

%s
%s

共 %d 卷"""
volume = entry["volume"]
dynasty_en = entry["dynasty_en"]
translator_en = entry["translator_en"]
nscrolls = entry["nscrolls"]
compiledby_en = entry["compiledby_en"]
compiled_how_en = entry["compiled_how_en"]
explainedby_en = entry["explainedby_en"]
attribution_cn = entry["attribution_cn"]
how_en = entry["how_en"]
colophon_cn = colophon_cn % (volume, tid, title_cn, attribution_cn, nscrolls)
#(volume, tid, nscrolls, translator_en, dynasty) = curation_util.ExtractFromColophon(colophon_cn)
kid = getkoreanid(tid)

pali = u""
japanese = u""

# References
abc = u"ABC '%s'" % pinyin
bl = u"BL '%s'" % english
bscd = u"BCSD '%s'" % traditional
ccd = u"CCD '%s'" % simplified
fe = u"FE '%s'" % traditional
fgdb = u"FGDB '%s'" % traditional
gced = u"GCED '%s'" % traditional[0]
ghdc = u"GHDC '%s'" % simplified
jebd = u"JEBD '%s'" % japanese
kdc = u"KDC %s" % kid
mw = u"MW '%s'" % sanskrit
ncced = u"NCCED '%s'" % simplified
t = u"T %s" % tid
if kid != "0":
  refArr = [fgdb, kdc, t] # Add based on references checked
else:
  refArr = [t]

# Generated
if grammar != "proper noun":
  pinyin = pinyin.replace(" ", "")

# Sanskrit, Pali, and Japanese equivalents
fromLang = ""
if sanskrit != "":
  fromLang = u"From Sanskrit: %s" % sanskrit
if pali != "":
  fromLang += u", Pali: %s" % pali
if japanese != "":
  fromLang += u", Japanese: %s" % japanese
if fromLang != "":
  fromLang += u"; "

# References
ref = u""
for r in refArr:
  if r != "":
    ref += r + "; "
ref = re.sub("; $", "", ref)
if ref != "":
  ref = u"(%s)" % ref

note = u"the name of a bodhisattva who is central in the 《阿差末菩薩經》Akṣayamatinirdeśasūtra in the Chinese Buddhist canon "

notes = u"%s%s%s" % (fromLang, note, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)

49829	澄观	澄觀	Chéng Guān	Cheng Guan	proper noun	经疏	Sūtra Commentary	佛教	Buddhism	中国佛教	Chinese Buddhism	\N	\N	the name of a bodhisattva who is central in the 《阿差末菩薩經》Akṣayamatinirdeśasūtra in the Chinese Buddhist canon (T 1735)	49829


In [38]:
# Template for a dictionary entry for a monastic
concept = u"法师\tMonastic"
#concept = u"翻译\tTranslator"
#concept = u"作者\tWriter"
#concept = u"\\N\t\\N"
daterange = u"738-839"
notes = u"%s, Chinesse Buddhist monastic from the 華嚴宗 Huayan school, credited with speaking the 《大方廣佛華嚴經疏》 Dafang Guang Fo Huayan Jing Shu, included in the Chinese Buddhist canon %s" % (daterange, ref)
#notes = u"See 寂友"
line = u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)
#curation_util.WriteWordEntry(line)
print line

49829	澄观	澄觀	Chéng Guān	Cheng Guan	proper noun	法师	Monastic	佛教	Buddhism	中国佛教	Chinese Buddhism	\N	\N	738-839, Chinesse Buddhist monastic from the 華嚴宗 Huayan school, credited with speaking the 《大方廣佛華嚴經疏》 Dafang Guang Fo Huayan Jing Shu, included in the Chinese Buddhist canon (T 1735)	49829


In [33]:
# Add a title from the Taisho to the dictionary and corpus
# Template for a Taisho title as a word entry in the dictionary

# Input
daterange = u"784-787"
textFormat = u"śastra"
#textFormat = u"Dhāraṇī"
genre = u"commentary"

# Generated    
if daterange == "":
  dateranges = {"Later Han": "25-220",
                "Eastern Jin": "317-420",
                "Northern Wei": "386-557",
                "Liang": "502-557",
                "Liu Song": "420-479",
                "Sui": "589-618",
                "Tang": "618-907",
                "Song": "960-1279",
                "Yuan": "1279-1368",
                "Ming": "1368-1644",
                "Qing": "1644-1911",
                "Korean Goryeo": "918-1392"}
  if dynasty_en in dateranges:
    daterange = dateranges[dynasty_en]
url = geturl(volume, tid)

# Add a separate entry if the title begins with 佛說
if newWord and traditional[:2] == u"佛說":
  trad = traditional[2:]
  notes = u"See %s" % trad
  line = u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, traditional, pinyin, english, grammar, concept, domain, subdomain, notes, luid)
  curation_util.WriteWordEntry(line)
  print line
  luid += 1
  simplified = simplified[2:]
  pinyin = pinyin.replace(u"Fó shuō ", "").strip()
  english = english.replace(u"Fo Shuo ", "").strip()

the = u"the"
if fromLang == "":
  the = u"The"
notes = u"%s%s name of a text in the Chinese Buddhist canon %s" % (
    fromLang, the, ref)
if newWord:
  line = u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
      luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)
  curation_util.WriteWordEntry(line)
  print line

# Full title of the Taisho entry
tidStr = tid
if tid < 1000:
  tidStr = "0%s" % tid
title = u"%s 《%s》" % (title_en, traditional)
title_html = u"          <tr>\n"
title_html += u"            <td>%s</td>\n" % tid
title_html += u"            <td><a href='/taisho/t%s.html'>%s</a></td>\n" % (tidStr, title)
title_html += u"          </tr>\n\n"
title_html += u"          {{next_title}}"
curation_util.InsertIntoVolume(volume, title_html)

# Collection entry
curation_util.WriteCollectionEntry(tid, title, translator_en, daterange, genre, how_en,
                                   textFormat, compiledby_en, compiled_how_en, explainedby_en)

# Write list of scrolls to metadata file
scrollsDF = pd.DataFrame({"i": range(1, nscrolls + 1)})
genSource = lambda i: "taisho/t%s_0%s.txt" % (tidStr, i) if (i < 10) else "taisho/t%s_%s.txt" % (tidStr, i)
scrollsDF['source'] = scrollsDF['i'].map(genSource)
genOut = lambda i: "taisho/t%s_0%d.html" % (tidStr, i) if (i < 10) else "taisho/t%s_%d.html" % (tidStr, i)
scrollsDF['output'] = scrollsDF['i'].map(genOut)
genTitle = lambda i: "Scroll %d" % i
scrollsDF['title'] = scrollsDF['i'].map(genTitle)
filename = "../data/corpus/taisho/t%s.csv" % tidStr 
cols = ['source', 'output', 'title']
scrollsDF.to_csv(filename, sep = "\t", header = False, columns = cols, index = False)

# Translation of colophon
print "compiled_how_en: %s by %s" % (compiled_how_en, compiledby_en)
curation_util.WriteColophon(tid, colophon_cn, volume, title_en, traditional, url, nscrolls, kid, sanskrit, 
              translator_en, dynasty_en, daterange, compiledby_en, how_en, compiled_how_en, explainedby_en)
saveScrolls(volume, tid, nscrolls, traditional)

Wrote line to words file
49828	大方广佛华严经疏	大方廣佛華嚴經疏	Dàfang Guǎng Fó Huáyán Jīng Shū	Dafang Guang Fo Huayan Jing Shu	proper noun	经疏	Sūtra Commentary	佛教	Buddhism	中国佛教	Chinese Buddhism	\N	\N	The name of a text in the Chinese Buddhist canon (T 1735)	49828
compiled_how_en:  by 
Writing colophon to ../corpus/taisho/t1735_00.txt
WriteColophon, translator = '澄觀'
saveScrollFromWeb, title: 大方廣佛華嚴經疏
http://tripitaka.cbeta.org/T35n1735_001
juanname' '大方廣佛華嚴經疏卷第一', tidStr = '1735'
start, juanname: 大方廣佛華嚴經疏卷第一, m.group(2): (并序)
saveScrollFromWeb, title: 大方廣佛華嚴經疏
http://tripitaka.cbeta.org/T35n1735_002
juanname' '大方廣佛華嚴經疏卷第二', tidStr = '1735'
start, juanname: 大方廣佛華嚴經疏卷第二, m.group(2): 
saveScrollFromWeb, title: 大方廣佛華嚴經疏
http://tripitaka.cbeta.org/T35n1735_003
juanname' '大方廣佛華嚴經疏卷第三', tidStr = '1735'
start, juanname: 大方廣佛華嚴經疏卷第三, m.group(2): 
saveScrollFromWeb, title: 大方廣佛華嚴經疏
http://tripitaka.cbeta.org/T35n1735_004
juanname' '大方廣佛華嚴經疏卷第四', tidStr = '1735'
start, juanname: 大方廣佛華嚴經疏卷第四, m.group(2): 
saveSc